# Overview

## Goal

## Problem Space

## Solution Space

## Approach

# Workbench

## Dataframes and ML

## Geometry and Model

## Mesh

## CFD

## Analysis

## Optimization

# Problem Setup

How do we set up our model and simulations, then measure things about the result?


## parameterized paper airplane design

###  gmsh - create parameterized plane design

### gmsh - buidl mesh

## 3D flow in air at low velocity

### SU2 run simulation

### Measure outcomes



# Solution Setup

How do we explore the design space and optimize some dependent variable output?

## simple uniform meshing

## simple CFD run

## metrics and efficacy

## optimization

# Implementation

How do we explore this problem?

## dataframes for CFD explorations and DoE

## Problem specification 
What is an interesting AoA to optimize along?

What are useful physical parameters?

Do our result make physical sense?


## optimization

How should we define fitness?

How should we optimize fitness with our single variable?

# Results

## Do our CFD results seem reasonable?

## Does our optimization seem reasonable?

## How does clever optimization compare to a grid search?


# Open Questions and Next Steps

* Optimization tools (Dakota, matlab, etc)
* CFD tooling
  * cfd params
  * mesh improvements
  * other CFD platforms
* problem refinement
* more parameter dimensions
* V & V